# COVID-19 Status in Morocco
> COVID-19 overview in Morocco

- author: <a href=https://github.com/Otman404/>Otman404</a>
- comments: true
- categories: [overview,interactive,plotly,Morocco]
- image: images/covid_Morocco.png
- permalink: /covid-overview-morocco/

In [108]:
#hide
#importing libraries

#storing and anaysis
import pandas as pd
import numpy as np

#visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import folium 
from folium import plugins
from IPython.display import HTML
plt.rcParams['figure.figsize'] = 10, 12
%matplotlib inline

# hide warnings
import warnings
warnings.filterwarnings('ignore')

In [109]:
#hide
# importing datasets
MA_data = pd.read_csv('https://raw.githubusercontent.com/aboullaite/Covid19-MA/master/stats/MA-times_series.csv')
by_cities = pd.read_csv('https://raw.githubusercontent.com/aboullaite/Covid19-MA/master/stats/cities.csv')
by_regions = pd.read_csv('https://raw.githubusercontent.com/aboullaite/Covid19-MA/master/stats/regions.csv')
# global_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
# global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
# global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [110]:
#hide
MA_data.head()

,Dates / التواريخ,Cases / الحالات,Recovered / تعافى,Deaths / الوفيات
0,02/03/2020,1,0,0
1,05/03/2020,2,0,0
2,10/03/2020,2,0,1
3,11/03/2020,6,0,1
4,13/03/2020,8,1,1


In [111]:
#hide
by_cities.head()

,City / المدينة,Region / المنطقة,Active Cases / الحالات النشطة,Total Deaths / إجمالي الوفيات,Total Recovered / إجمالي المعافين
0,Aïn Harrouda,Casablanca-Settat,0,0,0
1,Ben Yakhlef,Casablanca-Settat,0,0,0
2,Bouskoura,Casablanca-Settat,0,0,0
3,Casablanca,Casablanca-Settat,130,1,1
4,Médiouna,Casablanca-Settat,0,0,0


In [112]:
#hide
by_regions

,Region / الجهة,Total Cases / إجمالي الحالات,Active Cases / الحالات النشطة,Total Deaths / إجمالي الوفيات,Total Recovered / إجمالي المعافين
0,Tanger-Tétouan-Al Hoceima,917,723,24,170
1,Oriental,176,81,8,87
2,Fès-Meknès,896,807,26,63
3,Rabat-Salé-Kénitra,584,338,12,234
4,Béni Mellal-Khénifra,99,82,7,10
5,Casablanca-Settat,1752,1382,46,324
6,Marrakech-Safi,1213,1112,50,51
7,Drâa-Tafilalet,584,552,4,28
8,Souss-Massa,71,51,7,13
9,Guelmim-Oued Noun,43,42,0,1


In [113]:
#hide
# renaming columns
MA_data.columns = ['Dates','Confirmed','Recovered','Deaths']
by_cities.columns = ['City','Region','Confirmed','Deaths','Recovered']
by_regions.columns = ['Region','Total Cases','Confirmed','Deaths','Recovered']

In [114]:
#hide
# adding latitude and logtitude data for Moroccan regions to 'by_regions' dataframe
Latitudes = [35.3174,33.3260,33.7560,34.0980,32.4843,33.1010,31.8310,31.3860,29.7453,28.3870,26.7270,22.8170]
Longitudes = [-5.4657,-2.4220,-4.8450,-6.3280,-6.3199,-7.8060,-8.3170,-5.5470,-8.1436,-10.0910,-11.6610,-14.3610]
by_regions['Latitude'] = Latitudes
by_regions['Longitude'] = Longitudes

In [115]:
#hide
# remove frensh accents from data
cols = by_regions.select_dtypes(include=[np.object]).columns
by_regions[cols] = by_regions[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
by_regions

,Region,Total Cases,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Tanger-Tetouan-Al Hoceima,917,723,24,170,35.3174,-5.4657
1,Oriental,176,81,8,87,33.3260,-2.4220
2,Fes-Meknes,896,807,26,63,33.7560,-4.8450
3,Rabat-Sale-Kenitra,584,338,12,234,34.0980,-6.3280
4,Beni Mellal-Khenifra,99,82,7,10,32.4843,-6.3199
5,Casablanca-Settat,1752,1382,46,324,33.1010,-7.8060
6,Marrakech-Safi,1213,1112,50,51,31.8310,-8.3170
7,Draa-Tafilalet,584,552,4,28,31.3860,-5.5470
8,Souss-Massa,71,51,7,13,29.7453,-8.1436
9,Guelmim-Oued Noun,43,42,0,1,28.3870,-10.0910


In [116]:
#hide_input
total_confirmed = MA_data['Confirmed'].max()
total_recovered = MA_data['Recovered'].max()
total_deaths = MA_data['Deaths'].max()

from datetime import datetime

HTML(f'<h2>{datetime.today().strftime("%d-%m-%Y")} stats in Morocco: </h2><p style="color:#FFA800;"><span style="font-weight:bold;">Confirmed:</span> {total_confirmed}</p><p style="color:#18D26B;"><span style="font-weight:bold;">Recovered:</span> {total_recovered}</p><p style="color:#FF3F3F;"><span style="font-weight:bold;">Deaths:</span> {total_deaths}</p>')

## Regions with most  cases

In [117]:
#hide_input

by_regions.iloc[:,:-2].style.background_gradient(cmap='Reds')

,Region,Total Cases,Confirmed,Deaths,Recovered
0,Tanger-Tetouan-Al Hoceima,917,723,24,170
1,Oriental,176,81,8,87
2,Fes-Meknes,896,807,26,63
3,Rabat-Sale-Kenitra,584,338,12,234
4,Beni Mellal-Khenifra,99,82,7,10
5,Casablanca-Settat,1752,1382,46,324
6,Marrakech-Safi,1213,1112,50,51
7,Draa-Tafilalet,584,552,4,28
8,Souss-Massa,71,51,7,13
9,Guelmim-Oued Noun,43,42,0,1


In [118]:
#hide_input
fig = px.pie(by_regions, values='Confirmed', names='Region', title="COVID-19 Cases distribution",color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

## Total Active Cases in regions

In [119]:
#hide_input

total_active_cases_regions = pd.DataFrame({'Region':by_regions['Region'],'Active Cases':by_regions['Confirmed']})
total_active_cases_regions.style.background_gradient(cmap='Reds')

,Region,Active Cases
0,Tanger-Tetouan-Al Hoceima,723
1,Oriental,81
2,Fes-Meknes,807
3,Rabat-Sale-Kenitra,338
4,Beni Mellal-Khenifra,82
5,Casablanca-Settat,1382
6,Marrakech-Safi,1112
7,Draa-Tafilalet,552
8,Souss-Massa,51
9,Guelmim-Oued Noun,42


## Currently Infected

In [120]:
#hide_input
currently_infected = MA_data['Confirmed']-(MA_data['Recovered'] + MA_data['Deaths'])

fig = go.Figure(data=go.Scatter(x=MA_data['Dates'],y=currently_infected.values,mode='lines+markers'))
fig.update_layout(title='Active Cases in Morocco (Number of Infected people)',
                   xaxis_title='Days',
                   yaxis_title='New Cases')
fig.show()

In [121]:
#hide
regions_coord = by_regions[['Region','Latitude','Longitude']].copy()
regions_coord

,Region,Latitude,Longitude
0,Tanger-Tetouan-Al Hoceima,35.3174,-5.4657
1,Oriental,33.3260,-2.4220
2,Fes-Meknes,33.7560,-4.8450
3,Rabat-Sale-Kenitra,34.0980,-6.3280
4,Beni Mellal-Khenifra,32.4843,-6.3199
5,Casablanca-Settat,33.1010,-7.8060
6,Marrakech-Safi,31.8310,-8.3170
7,Draa-Tafilalet,31.3860,-5.5470
8,Souss-Massa,29.7453,-8.1436
9,Guelmim-Oued Noun,28.3870,-10.0910


In [122]:
#hide
by_regions

,Region,Total Cases,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Tanger-Tetouan-Al Hoceima,917,723,24,170,35.3174,-5.4657
1,Oriental,176,81,8,87,33.3260,-2.4220
2,Fes-Meknes,896,807,26,63,33.7560,-4.8450
3,Rabat-Sale-Kenitra,584,338,12,234,34.0980,-6.3280
4,Beni Mellal-Khenifra,99,82,7,10,32.4843,-6.3199
5,Casablanca-Settat,1752,1382,46,324,33.1010,-7.8060
6,Marrakech-Safi,1213,1112,50,51,31.8310,-8.3170
7,Draa-Tafilalet,584,552,4,28,31.3860,-5.5470
8,Souss-Massa,71,51,7,13,29.7453,-8.1436
9,Guelmim-Oued Noun,43,42,0,1,28.3870,-10.0910


In [123]:
#hide_input

fig = go.Figure()
fig.add_trace(go.Bar(
    y=by_regions['Region'],
    x=by_regions['Confirmed'],
    name='Confirmed',
    orientation='h',
    marker=dict(
        color='rgba(222, 25, 27, 0.6)',
        line=dict(color='rgba(222, 25, 27, 0.6)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=by_regions['Region'],
    x=by_regions['Recovered'],
    name='Recovered',
    orientation='h',
    marker=dict(
        color='rgba(80, 180, 50, 0.6)',
        line=dict(color='rgba(80, 180, 50, 0.6)', width=3)
    )
))

fig.update_layout(barmode='stack',
                  title='COVID-19 Cases in Moroccan Regions',
                   xaxis_title='Cases',
                   yaxis_title='Regions')

fig.show()

In [124]:
#hide_input
fig = go.Figure()

fig.add_trace(go.Scatter(x=MA_data['Dates'], y=MA_data['Confirmed'], name='Confirmed',
                         line=dict(color='royalblue', width=4)))
fig.add_trace(go.Scatter(x=MA_data['Dates'], y=MA_data['Recovered'], name ='Recovered',
                         line=dict(color='limegreen', width=4)))
fig.add_trace(go.Scatter(x=MA_data['Dates'], y=MA_data['Deaths'], name='Deaths',
                         line=dict(color='firebrick', width=4,
                              dash='dash')
))


fig.update_layout(title='Total Coronavirus Cases in Morocco',
                   xaxis_title='Days',
                   yaxis_title='Total Cases')
fig.show()

In [125]:
#hide
new_confirmed_cases = [0]
new_recovered_cases = [0]
new_death_cases = [0]

for i,r in MA_data[['Confirmed','Recovered','Deaths']].iterrows():
    if i > 0:
        new_confirmed_cases.append(r['Confirmed'] - MA_data['Confirmed'][i-1])
        new_recovered_cases.append(r['Recovered'] - MA_data['Recovered'][i-1])
        new_death_cases.append(r['Deaths'] - MA_data['Deaths'][i-1])
#hide
new_cases_df = pd.DataFrame({'Dates':MA_data['Dates'],'new_confirmed':new_confirmed_cases,'new_recovered':new_recovered_cases,'new_deaths':new_death_cases})
new_cases_df

,Dates,new_confirmed,new_recovered,new_deaths
0,02/03/2020,0,0,0
1,05/03/2020,1,0,0
2,10/03/2020,0,0,1
3,11/03/2020,4,0,0
4,13/03/2020,2,1,0
5,14/03/2020,10,0,0
6,15/03/2020,11,0,0
7,16/03/2020,9,0,1
8,17/03/2020,6,0,0
9,18/03/2020,10,0,0


In [126]:
#hide_input

# data
df_long=pd.melt(new_cases_df, id_vars=['Dates'], value_vars=['new_confirmed','new_deaths','new_recovered'])

# plotly 
fig = px.line(df_long, x='Dates', y='value', color='variable')
fig.update_layout(title='Daily New Cases in Morocco',
                   xaxis_title='Days',
                   yaxis_title='New Cases')
# Show plot 
fig.show()

In [127]:
#hide_input

# Confirmed Cases
fig = px.bar(MA_data,x="Dates",y="Confirmed",color="Confirmed",orientation="v",
             height=600,title="Confirmed",
             color_discrete_sequence=px.colors.cyclical.IceFire)
#fig.update_layout(plot_bgcolor='rgb(230,230,230)')
fig.show()

In [128]:
#hide_input
# Recovered Cases
fig = px.bar(MA_data,x="Dates",y="Recovered",color="Recovered",orientation="v",
             height=600,title="Recovered",
             color_discrete_sequence=px.colors.sequential.Plasma)
#fig.update_layout(plot_bgcolor='rgb(230,230,230)')
fig.show()

In [129]:
#hide_input
# Deaths Cases
fig = px.bar(MA_data,x="Dates",y="Deaths",color="Deaths",orientation="v",
             height=600,title="Deaths",
             color_discrete_sequence=px.colors.cyclical.mrybm)
#fig.update_layout(plot_bgcolor='rgb(230,230,230)')
fig.show()